In [1]:
import pandas as pd
import xml.etree.cElementTree as ET
from datetime import datetime
import urllib
from pprint import pprint

In [2]:
categories = pd.read_excel('categories.xlsx', names=['category', 'cat_id'])

categories_dict=(categories.set_index('category').to_dict())['cat_id']

def cat2id(cat_name):
   return categories_dict[cat_name]

In [3]:
file = pd.read_excel('подборка для IMR.xlsx')

In [4]:
img_links = []

now = datetime.strftime(datetime.now(), "%Y-%m-%d %H:%M")
yml = ET.Element('yml_catalog', date = str(now))
shop = ET.SubElement(yml, "shop")

ET.SubElement(shop, 'name').text = 'Homefash-Trading'
ET.SubElement(shop, 'company').text = 'ООО МОЙ ДОМ'
ET.SubElement(shop, 'url').text = 'http://homefash-trading.com/'

ET.SubElement(ET.SubElement(shop, 'currencies'), 'currency', id='RUR', rate='1')

categories_xml = ET.SubElement(shop, 'categories')
for cat in categories_dict:
    if len(str(categories_dict[cat]))==4:
        ET.SubElement(categories_xml, 'category', id = str(categories_dict[cat]), parentId=str(categories_dict[cat])[:2]).text = cat
    else:
        ET.SubElement(categories_xml, 'category', id = str(categories_dict[cat])).text = cat

In [5]:
offers = ET.SubElement(shop, "offers")
i=0

def add_offers(file=file, i=i):
    
    def sub_lmnt(name, column, i=i):
        try:
            ET.SubElement(offer, name).text = str(file.ix[i][column])
        except KeyError:
            pass
        
    while (i<len(file.index)):
        if int(file.ix[i]['Warehouse Qty']) != 0:
        #     offer = ET.SubElement(offers, "offer")
            offer = ET.SubElement(offers, "offer", id=str(file.ix[i]['Homefash SKU']), available='true')
            sub_lmnt('price', 'RRP', i)
            ET.SubElement(offer, 'currencyId').text = 'RUR'
            sub_lmnt('name', 'Product Name', i)
        #     sub_lmnt('description', 'Full Product Description', i)
    #         sub_lmnt('description', 'Full Product Description', i)

    #         print(file.ix[i]['Full Product Description'])



            if str(file.ix[i]['Full Product Description']) != 'nan':
                sub_lmnt('description', 'Full Product Description', i)
            elif str(file.ix[i]['Material']) != 'nan':
                sub_lmnt('description', 'Material', i)
            elif str(file.ix[i]['Description']) != 'nan':
                sub_lmnt('description', 'Description', i)

            sub_lmnt('vendor', 'Brand', i)
            sub_lmnt('vendorCode', 'Product Number', i)
            sub_lmnt('dimensions', 'Product Dimensions', i)
        #     picture
            link = r'http://u0256746.isp.regruhosting.ru/images/'+ str(file.ix[i]['Brand']) +'/' + str(file.ix[i]['Photoname']) + '.jpg'
            ET.SubElement(offer, 'picture').text = link
            img_links.append(link)
            try:
                ET.SubElement(offer, 'categoryId').text = str(cat2id(file.ix[i]['Category B']))
            except KeyError:
                ET.SubElement(offer, 'categoryId').text = str(cat2id(file.ix[i]['Category A']))     
        #     sub_lmnt('dimensions', '')
            sub_lmnt('country_of_origin', 'Country of Origin', i)
            sub_lmnt('stock', 'Warehouse Qty', i)
            sub_lmnt('weight', 'Product Weight, kg', i)

        i += 1

In [6]:
add_offers()

In [7]:
tree = ET.ElementTree(yml)
tree.write("IMR.xml", encoding="UTF-8", xml_declaration=True)

In [8]:
error_list = []
for img in img_links:
    try:
        dwnld = urllib.request.urlopen(img).read()
    except BaseException:
        error_list.append(img)

In [9]:
for error in error_list:
    print(error)

http://u0256746.isp.regruhosting.ru/images/Broste/14533030.jpg
http://u0256746.isp.regruhosting.ru/images/Stelton/z00104.jpg
http://u0256746.isp.regruhosting.ru/images/Stelton/z00055.jpg
http://u0256746.isp.regruhosting.ru/images/Stelton/z00120.jpg
http://u0256746.isp.regruhosting.ru/images/Stelton/z00121.jpg
http://u0256746.isp.regruhosting.ru/images/Stelton/z00122.jpg


In [12]:
import ftplib

server = '37.140.192.162'
username = 'u0193818'
password = 'zQK!p9wW'
ftp_connection = ftplib.FTP(server, username, password)

remote_path = "/www/homefash-trading.com/xml_export/"
ftp_connection.cwd(remote_path)

fh = open("IMR.xml", 'rb')
ftp_connection.storbinary('STOR IMR.xml', fh)
fh.close()